In [1]:
import numpy as np

In [2]:
class Tensor:
    """
    Tensor
    """
    def __init__(self, numpy_array):
        self.value = numpy_array
        self.previous_tensor = None
        self.operation = None
        self.grad = None
        
    def backward(self):
        # check
        if not self.value.size == 1:
            raise ValueError("The tensor you are trying to take derivative must be a scalar.\
                             (For example, you should call `.backward()` after `.mean` operation)")
        
        p_ahead = self
        p = self.previous_tensor
        grad = None
        while p is not None:
            if p.operation[0] == 'add':
                if grad is None:
                    grad = 1
                else:
                    grad = grad
            elif p.operation[0] == 'sub':
                if grad is None:
                    grad = 1
                else:
                    grad = grad
            elif p.operation[0] == 'rsub':
                if grad is None:
                    grad = -1
                else:
                    grad = grad * -1
            elif p.operation[0] == 'mul':
                if grad is None:
                    grad = p.operation[1]
                else:
                    grad = grad * p.operation[1]
            elif p.operation[0] == 'mean':
                if grad is None:
                    grad = 1.0 / p.value.size
                else:
                    grad = grad * (1.0 / p.size)
            else:
                raise ValueError("Unknown operation")
            print('grad:', grad)
            p = p.previous_tensor
            p_ahead = p_ahead.previous_tensor
        p_ahead.grad = grad
        
    def __add__(self, other):
        if isinstance(other, int) or isinstance(other, float):
            self.operation = ('add', other)
            operation_result = self.value + other
            new_tensor = Tensor(numpy_array=operation_result)
            new_tensor.previous_tensor = self
            return new_tensor
        elif isinstance(other, np.ndarray):
            # check sizes
            pass
        elif isinstance(other, Tensor):
            pass
        else:
            raise ValueError("Invalid operand type for {}. type: {}".\
                                 format(str(other), str(type(other))))
        
    
    def __radd__(self, other):
        return self.__add__(other)
    
    def __sub__(self, other):
        self.operation = ('sub', other)
        operation_result = self.value - other
        new_tensor = Tensor(numpy_array=operation_result)
        new_tensor.previous_tensor = self
        return new_tensor
        
    def __rsub__(self, other):
        self.operation = ('rsub', other)
        operation_result = other - self.value
        new_tensor = Tensor(numpy_array=operation_result)
        new_tensor.previous_tensor = self
        return new_tensor
    
    def __mul__(self, other):
        self.operation = ('mul', other)
        operation_result = self.value * other
        new_tensor = Tensor(numpy_array=operation_result)
        new_tensor.previous_tensor = self
        return new_tensor
    
    def mean(self):
        self.operation = ('mean', None)
        operation_result = self.value.mean()
        new_tensor = Tensor(numpy_array=operation_result)
        new_tensor.previous_tensor = self
        return new_tensor
    
    def __str__(self):
        return self.value.__repr__()